In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from chat_dataset import ChatDataset
from chat_dataloader import ChatDataLoader
import torch
from torch.utils.data import DataLoader
import pandas as pd
import argparse

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [7]:
model_name = 'skt/ko-gpt-trinity-1.2B-v0.5'
file_path = '/home/yys/COMU-ChatBot/data_final.tsv'
output_dir = '/home/yys/COMU-ChatBot/gpt/output'

In [14]:
# 데이터셋 및 모델 로딩
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [11]:
def gpt_train(args):
    # model  = AutoModelForCausalLM.from_pretrained(args.model_checkpoint)
    # tokenizer = AutoTokenizer.from_pretrained(args.model_checkpoint, use_fast=True, stride=128)
    
    model.config.max_length = args.max_target_length
    tokenizer.model_max_length = args.max_target_length
    return model, tokenizer

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Hyperparameter for Training gpt-based model for conditional generation')
    parser.add_argument('--model_checkpoint', default='skt/ko-gpt-trinity-1.2B-v0.5', type=str,
                        help='huggingface model name to train')
    parser.add_argument('--max_input_length', default=100, type=int,
                        help='max input length for dialog')
    parser.add_argument('--max_target_length', default=100, type=int,
                    help='max target length for dialog')
    parser.add_argument('--train_batch_size', default=8, type=int,
                                            help='train batch size')
    parser.add_argument('--num_train_epochs', default=1, type=int,
                                            help='train epoch size')
    parser.add_argument('--lr', default=5e-5, type=int,
                                            help='learning rate for training')
    parser.add_argument('--wd', default=0.01, type=int,
                                            help='weight decay for training'),
    parser.add_argument('--model_name', default='skt/ko-gpt-trinity-1.2B-v0.5', type=str,
                                            help='model name for saving')
    parser.add_argument('--base_path', default='../data/data_final.tsv', type=str,
                                                help='dataset path')
    parser.add_argument('--model_path', default='./output', type=str,
                                                help='model path for saving')

    args = parser.parse_args()
    
    #Load dataset
    dataset = pd.read_csv(args.base_path,sep='\t')
    tokenized_dataset = ChatDataset(dataset)
    batched_dataset = ChatDataset(tokenized_dataset, args.train_batch_size)
    model = AutoModelForCausalLM.from_pretrained(args.model_checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(args.model_checkpoint, use_fast=True, stride=128)
    
    training_args = TrainingArguments(
        output_dir = args.model_path,
        evaluation_strategy='epoch',
        save_total_limit=1,
        num_train_epochs=args.num_train_epochs,
        per_device_train_batch_size=args.train_batch_size,
        save_steps=1000,
        learning_rate=args.lr,
        weight_decay=args.wd,
        warmup_steps=3000,
        logging_steps=1000,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model='loss',
        greater_is_better=False     
    )
    
    trainer = Trainer(
        model = model,
        args=training_args,
        train_dataset=batched_dataset,
        data_collator=ChatDataLoader.collate_fn,
        tokenizer=tokenizer
    )
    # Training
    print('Start Training...')  
    
    trainer.train()
    
    # Saving model
    print('Saving Model...')
    trainer.save_model()

usage: ipykernel_launcher.py [-h] [--model_checkpoint MODEL_CHECKPOINT]
                             [--max_input_length MAX_INPUT_LENGTH]
                             [--max_target_length MAX_TARGET_LENGTH]
                             [--train_batch_size TRAIN_BATCH_SIZE]
                             [--num_train_epochs NUM_TRAIN_EPOCHS] [--lr LR]
                             [--wd WD] [--model_name MODEL_NAME]
                             [--base_path BASE_PATH] [--model_path MODEL_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/yys/.local/share/jupyter/runtime/kernel-8596848d-3d6c-4c30-8a6c-71bb3f74664d.json


SystemExit: 2

/home/yys/.conda/envs/chatbot-gpt/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
dataset = pd.read_csv('../data/data_final.tsv',sep='\t')
tokenized_dataset = ChatDataset(dataset)
batched_dataset = ChatDataLoader(tokenized_dataset, 4)
model = AutoModelForCausalLM.from_pretrained('skt/ko-gpt-trinity-1.2B-v0.5')
tokenizer = AutoTokenizer.from_pretrained('skt/ko-gpt-trinity-1.2B-v0.5', use_fast=True, stride=128)

In [22]:
for i in range(3):
    print(type(tokenized_dataset[0][i]), len(tokenized_dataset[0][i]))

<class 'list'> 50
<class 'numpy.ndarray'> 50
<class 'list'> 50


In [21]:
len(batched_dataset)

7692

In [8]:
for batch in train_dataloader:
    token_ids, mask, labels_ids = batch
    print("Token ids:", token_ids)
    print("Mask:", mask)
    print("Labels ids:", labels_ids)
    break

/home/yys/COMU-ChatBot/gpt/chat_dataloader.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
/home/yys/COMU-ChatBot/gpt/chat_dataloader.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


Token ids: tensor([[    2, 37215, 27248, 22912, 30930, 41095, 43781, 30195, 30021, 27212,
         30476, 35891, 20476, 18904, 33252,   407,     4, 31927, 25837, 20476,
         34327, 25768, 30143, 30021, 33333, 29979, 30086, 39999, 30930,     1,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 39032, 18808, 32730, 30086, 47631, 46923, 46435,   407,     4,
         30096, 24660, 37731, 29392,     1,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3]])
Mask: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 